In [ ]:
!pip install peft
!pip install --upgrade peft

In [ ]:
import pandas as pd
from transformers import AutoImageProcessor, AutoModelForImageClassification
from tqdm import tqdm
import torch
import torch.nn as nn
from tqdm import tqdm
from transformers import ResNetModel, ResNetConfig
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
import os
from PIL import Image
import torch
from torchvision import transforms
import os
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from torch import nn, optim
import peft
from peft import get_peft_model
from peft import LoraConfig
import torch
import torch.nn.functional as F

In [ ]:
!unzip /kaggle/input/galaxy-zoo-the-galaxy-challenge/training_solutions_rev1.zip

In [ ]:
!unzip /kaggle/input/galaxy-zoo-the-galaxy-challenge/images_training_rev1.zip

In [ ]:
!unzip /kaggle/input/galaxy-zoo-the-galaxy-challenge/images_test_rev1.zip

In [ ]:
df = pd.read_csv("/kaggle/working/training_solutions_rev1.csv")

In [ ]:
df.head()

In [ ]:
folder_path = "/kaggle/working/images_training_rev1" 

image_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith(('.png', '.jpg', '.jpeg'))]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class GalaxyZooDataset(Dataset):
    def __init__(self, data, images_folder, transform=None):
        self.data = data 
        self.images_folder = images_folder
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        galaxy_id = str(int(row['GalaxyID']))
        image_path = os.path.join(self.images_folder, f"{galaxy_id}.jpg")  
        
        if not os.path.exists(image_path):
            print(f"Image {image_path} not found.")
            image = Image.new('RGB', (224, 224), color=(255, 255, 255)) 
        else:
            image = Image.open(image_path).convert("RGB")

        if self.transform:
            image = self.transform(image)
        
        labels = torch.tensor(row[1:].values, dtype=torch.float)
        
        return image, labels

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

df = pd.read_csv("/kaggle/working/training_solutions_rev1.csv")

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)  # 80% train, 20% test

train_dataset = GalaxyZooDataset(data=train_df, images_folder="/kaggle/working/images_training_rev1", transform=transform)
test_dataset = GalaxyZooDataset(data=test_df, images_folder="/kaggle/working/images_training_rev1", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad(): 
        for images, labels in tqdm(dataloader):
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            
            probs = F.softmax(outputs, dim=1)
            
            _, predicted = torch.max(probs, 1)

            labels = labels.argmax(dim=1) 

            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total * 100
    return accuracy

### NORMAL FINE-TUNING

In [ ]:
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 37)

In [ ]:
model = model.to(device)
print("Accuracy before finetuning is : ",evaluate_model(model,test_loader))

In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
def train_model(model, dataloader, optimizer, num_epochs=2):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        print(f"Epoch {epoch+1}/{num_epochs}")
        
        for images, labels in tqdm(dataloader, desc=f"Training Epoch {epoch+1}", leave=False):
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        print(f"Loss: {running_loss / len(dataloader):.4f}")

train_model(model, train_loader, optimizer)

fine_tuned_loss = evaluate_model(model, test_loader)
print(f"Fine-tuned Model Accuracy: {fine_tuned_loss:.4f}")

In [ ]:
model.eval()
print("Accuracy of normal fine-tuning",evaluate_model(model,test_loader))

### LORA FINE-TUNING

In [ ]:
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 37) 

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=['fc'],
    lora_dropout=0.01,
)

model = get_peft_model(model, lora_config)
model = model.to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

def train_model_with_lora(model, dataloader, optimizer, num_epochs=2):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        print(f"Epoch {epoch+1}/{num_epochs}")
        
        for images, labels in tqdm(dataloader, desc=f"LoRA Fine-tuning Epoch {epoch+1}", leave=False):
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            # print(outputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        print(f"Loss: {running_loss / len(dataloader):.4f}")

train_model_with_lora(model, train_loader, optimizer)

lora_fine_tuned_loss = evaluate_model(model, test_loader)
print(f"LoRA Fine-tuned Model Accuracy: {lora_fine_tuned_loss:.4f}")

In [ ]:
model.eval()
print("Accuracy of lora fine-tuning",evaluate_model(model,test_loader))